In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/universal-sentence-encoder/tensorflow2/universal-sentence-encoder/2/saved_model.pb
/kaggle/input/universal-sentence-encoder/tensorflow2/universal-sentence-encoder/2/variables/variables.index
/kaggle/input/universal-sentence-encoder/tensorflow2/universal-sentence-encoder/2/variables/variables.data-00000-of-00001
/kaggle/input/universal-sentence-encoder/tensorflow2/cmlm-en-base/1/saved_model.pb
/kaggle/input/universal-sentence-encoder/tensorflow2/cmlm-en-base/1/assets/vocab.txt
/kaggle/input/universal-sentence-encoder/tensorflow2/cmlm-en-base/1/variables/variables.index
/kaggle/input/universal-sentence-encoder/tensorflow2/cmlm-en-base/1/variables/variables.data-00000-of-00001


In [2]:
!wget https://raw.githubusercontent.com/rvishakhs/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py

--2025-02-12 15:07:42--  https://raw.githubusercontent.com/rvishakhs/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2025-02-12 15:07:43 (77.2 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, walk_through_dir

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2025-02-12 15:07:56--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.207, 142.250.107.207, 108.177.98.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2025-02-12 15:07:57 (105 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
unzip_data('nlp_getting_started.zip')

In [6]:
import pandas as pd
import tensorflow as tf
import numpy as np


In [7]:
train_dataset = pd.read_csv('/kaggle/working/train.csv')
train_dataset.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
test_dataset = pd.read_csv('/kaggle/working/test.csv')
test_dataset.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
# Shuffle the data 
train_dataset_shuffled = train_dataset.sample(frac=1, random_state=42)
train_dataset_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [10]:
train_dataset_shuffled.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [11]:
# Display some random training samples 
import random
random_index = random.randint(0, len(train_dataset_shuffled)-5)
for row in train_dataset_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
    _, text, target = row
    print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text:\n{text}\n")
    print("----\n")

Target: 1 (real disaster)
Text:
Debris found on Reunion Island comes from MH370: Malaysian PM http://t.co/f75qWyeeEC

----

Target: 0 (not real disaster)
Text:
It's raining outside I'm burning my favorite candle and there's hot cocoa to sip. Nap time = awesomesauce.

----

Target: 0 (not real disaster)
Text:
@58hif my trick is to think about nasty things

----

Target: 0 (not real disaster)
Text:
Hunger hits me and I can't function probably wouldn't last long in famine zones***thank God***

----

Target: 0 (not real disaster)
Text:
Look for my Policy Matters Ohio report on #CLE and Cuyahoga County blight and greening vacant lands soon!  https://t.co/if62SdXVp7

----



In [12]:
# Split data into training and Validation data 
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_dataset_shuffled['text'].to_numpy(),
                                                                           train_dataset_shuffled['target'].to_numpy(),
                                                                           test_size=0.1, 
                                                                           random_state=42)

In [13]:
# Check the lengths
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [14]:
# Display first 10 training sentences with coressponding labels 
train_sentences[:10], train_labels[:10]


(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

# Text Vectorization (Tokenization)


In [15]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

text_vectorizer = TextVectorization(max_tokens=None, 
                                   split="whitespace",
                                   ngrams=None,
                                   output_mode="int",
                                   standardize='lower_and_strip_punctuation',
                                   output_sequence_length=None
                                   )

In [16]:
# Find average number of tokens (words) in training datasets 
round(sum([len(i.split()) for i in train_sentences]) / len(train_sentences))

15

In [17]:
# Setup Hyper parameters 
MAX_VOCAL_LENGTH = 10000 # max number of words to have in our vacabulary
MAX_LENGTH = 15 

text_vectorizer = TextVectorization(max_tokens=MAX_VOCAL_LENGTH,
                                    output_mode="int",
                                    output_sequence_length=MAX_LENGTH)

In [18]:
# Let's fit the text vectorozer to the training text
text_vectorizer.adapt(train_sentences)

In [19]:
# Creata a simple sentence and find out 
sample = "I love to read book and drive"
text_vectorizer([sample])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  8, 110,   5, 193, 747,   7, 955,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [20]:
# Choose random sentences from the training 
random_text = random.choice(train_sentences)
print(f"orginal Text: \n {random_text}")
print("Vectorized Version:  ")
text_vectorizer([random_text])

orginal Text: 
 HALSEY AND TROYE COLLAB WOULD BE BOMB
Vectorized Version:  


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1,    7, 7344,    1,   78,   21,  108,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [21]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_5_words = words_in_vocab[-5:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}") 
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [22]:
# Embedding 
from tensorflow.keras import layers 

embedding = layers.Embedding(input_dim=MAX_VOCAL_LENGTH,
                            output_dim=128,
                            )

embedding

<Embedding name=embedding, built=False>

In [23]:
random_text = random.choice(train_sentences)
print(f'Orginal text: {random_text}\
      \n Embedded Version: ')

# Embed the random sentences 
sample_embed = embedding(text_vectorizer([random_text]))
sample_embed 

Orginal text: Mum's literally just picked up her new car today and flattened the battery already trying to sort out the Bluetooth ???? #Muppet      
 Embedded Version: 


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.00699865,  0.03482871, -0.00909681, ..., -0.0052816 ,
          0.04331485,  0.00670005],
        [ 0.00335157,  0.0127886 , -0.03495526, ..., -0.01380805,
          0.0449584 ,  0.04105116],
        [ 0.02407131,  0.03939674,  0.00749598, ..., -0.01301996,
         -0.01435693, -0.04415988],
        ...,
        [ 0.0480918 , -0.04183561,  0.01466392, ...,  0.04984197,
         -0.02762347,  0.0081005 ],
        [-0.0053537 ,  0.0422218 , -0.04558622, ..., -0.04312735,
          0.0255428 , -0.03406294],
        [ 0.01049076, -0.02890481, -0.04423739, ...,  0.0422636 ,
          0.03312141, -0.01722682]]], dtype=float32)>

# Model building


In [24]:
# Create a base model with sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modeling pipeline
model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()), # converts words to numbers using tfidf
    ("clf", MultinomialNB()) # Model the text 
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [25]:
# Check the lengths
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [26]:
# Evalaute our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model acheives an accuracy of : {baseline_score * 100 :.2f} %")

Our baseline model acheives an accuracy of : 79.27 %


In [27]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Create a function to evaluate accuracy, precision, recall and F1 score 
def calculate_results(y_true, y_pred):
    """
    Calculates model accuracy, precision, recall, and f1score of a binary classification model.
    """

    # Calculate model accuracy
    model_accuracy = round(accuracy_score(y_true, y_pred) * 100, 2) 

    # Calculate model precision, recall and F1score using "weighted" average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
    model_results = {"Accuracy Score " : model_accuracy, 
                    "Precision Score" : model_precision,
                    "Recall Score" : model_recall,
                    "F1 Score " : model_f1}
    return model_results

In [28]:
baseline_predictions = model_0.predict(val_sentences)
baseline_predictions[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [29]:
# Calling the evaluation function
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_predictions)
baseline_results 

{'Accuracy Score ': 79.27,
 'Precision Score': 0.8111390004213173,
 'Recall Score': 0.7926509186351706,
 'F1 Score ': 0.7862189758049549}

In [30]:
from helper_functions import create_tensorboard_callback

# Create a directory to save Tensorboard logs 
SAVE_DIR = "model_logs"

In [31]:
# Build model with the functional API
from tensorflow.keras import layers 
inputs = layers.Input(shape=(1,), dtype=tf.string) 
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_1 = tf.keras.Model(inputs, outputs, name='model_1_Dense')

In [32]:
model_1.summary()

Model: "model_1_Dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [33]:
# Compile model 
model_1.compile(loss='binary_crossentropy',
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [34]:
# Fit the model 
model_1_history = model_1.fit(x=train_sentences,
                             y=train_labels,
                             epochs=5,
                             validation_data=(val_sentences, val_labels),
                             callbacks=[create_tensorboard_callback(dir_name='SAVE_DIR',
                                                                   experiment_name="Model_1_experiment")] )

Saving TensorBoard log files to: SAVE_DIR/Model_1_experiment/20250212-150802
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.6540 - loss: 0.6491 - val_accuracy: 0.7493 - val_loss: 0.5356
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8081 - loss: 0.4615 - val_accuracy: 0.7887 - val_loss: 0.4736
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8614 - loss: 0.3563 - val_accuracy: 0.7913 - val_loss: 0.4592
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8913 - loss: 0.2897 - val_accuracy: 0.7861 - val_loss: 0.4700
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9106 - loss: 0.2445 - val_accuracy: 0.7861 - val_loss: 0.4907


In [35]:
# Fit the model 
model_1_history = model_1.fit(x=train_sentences,
                             y=train_labels,
                             epochs=5,
                             validation_data=(val_sentences, val_labels),
                             callbacks=[create_tensorboard_callback(dir_name='SAVE_DIR',
                                                                   experiment_name="Model_1_experiment")] )

Saving TensorBoard log files to: SAVE_DIR/Model_1_experiment/20250212-150809
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9277 - loss: 0.2015 - val_accuracy: 0.7835 - val_loss: 0.5002
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9409 - loss: 0.1686 - val_accuracy: 0.7795 - val_loss: 0.5279
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9478 - loss: 0.1533 - val_accuracy: 0.7835 - val_loss: 0.5510
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9555 - loss: 0.1307 - val_accuracy: 0.7703 - val_loss: 0.5852
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9670 - loss: 0.1071 - val_accuracy: 0.7795 - val_loss: 0.6060


In [36]:
model_1.evaluate(val_sentences, val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7587 - loss: 0.6571 


[0.6060349941253662, 0.7795275449752808]

In [37]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


array([[0.5547853 ],
       [0.73883545],
       [0.999637  ],
       [0.072478  ],
       [0.01216272],
       [0.9832908 ],
       [0.89383763],
       [0.99961877],
       [0.9985954 ],
       [0.45799604]], dtype=float32)

In [38]:
# Converts model prediction probabilities to label format 
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [39]:
# Calculating actual results 
model_1_results = calculate_results(y_true=val_labels,
                                   y_pred=model_1_preds)
model_1_results

{'Accuracy Score ': 77.95,
 'Precision Score': 0.7802422774076316,
 'Recall Score': 0.7795275590551181,
 'F1 Score ': 0.7781078501550943}

In [40]:
# Visaualize Learned Embedding 
# Get thevocab from the text vectorization layer 
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [41]:
# Get the weights of our embedding layer 
embed_weights = model_1.get_layer("embedding").get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [42]:
# Creating embedding files to display them on the tensorflow projector 
import io

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()


In [43]:
# Create a LSTM Model
from tensorflow.keras import layers 
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(64, return_sequences = True)(x)
x = layers.LSTM(64)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs,outputs, name="Model_2_LSTM")

In [44]:
# Check the summary
model_2.summary()

Model: "Model_2_LSTM"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 15, 64)              │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,362,497 (5.20 MB)

 Trainable params: 1,362,497 (5.20 MB)

 Non-trainable params: 0 (0.00 B)

In [45]:
# Compile the model 
model_2.compile(loss='binary_crossentropy',
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [46]:
# Fit the model 
model_2_history = model_2.fit(train_sentences,
                             train_labels,
                             epochs=5,
                             validation_data=(val_sentences, val_labels),
                             callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                                   "Model_2_LSTM")])


Saving TensorBoard log files to: model_logs/Model_2_LSTM/20250212-150816
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9272 - loss: 0.2237 - val_accuracy: 0.7690 - val_loss: 0.8155
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9694 - loss: 0.0905 - val_accuracy: 0.7690 - val_loss: 0.8756
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9724 - loss: 0.0748 - val_accuracy: 0.7638 - val_loss: 0.8763
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9747 - loss: 0.0618 - val_accuracy: 0.7690 - val_loss: 1.2471
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9769 - loss: 0.0617 - val_accuracy: 0.7677 - val_loss: 1.0105


In [47]:
# Evaluate the model 
evaluate = model_2.evaluate(val_sentences, val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7407 - loss: 1.1140


In [48]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:20]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step


array([[1.3816449e-01],
       [6.8725032e-01],
       [9.9977320e-01],
       [1.2499644e-01],
       [1.0264195e-04],
       [9.9851948e-01],
       [6.2564886e-01],
       [9.9991524e-01],
       [9.9988115e-01],
       [7.8863621e-01],
       [1.5192419e-04],
       [8.9962363e-01],
       [1.1866468e-04],
       [1.2428566e-01],
       [6.8244837e-05],
       [1.2579575e-02],
       [6.0779200e-04],
       [8.0889103e-04],
       [8.0780961e-02],
       [9.9899715e-01]], dtype=float32)

In [49]:
# Converts model prediction probabilities to label format 
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [50]:
# Calculating actual results 
model_2_results = calculate_results(y_true=val_labels,
                                   y_pred=model_2_preds)
model_2_results

{'Accuracy Score ': 76.77,
 'Precision Score': 0.768489862704666,
 'Recall Score': 0.7677165354330708,
 'F1 Score ': 0.7660466459325422}

In [51]:
# Model 3 GRU 

from tensorflow.keras import layers 
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs,outputs, name="Model_3_GRU")

In [52]:
# Compile the model 
model_3.compile(loss='binary_crossentropy',
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [53]:
# Fit the model 
model_3_history = model_3.fit(train_sentences,
                             train_labels,
                             epochs=5,
                             validation_data=(val_sentences, val_labels),
                             callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                                   "Model_3_GRU")])


Saving TensorBoard log files to: model_logs/Model_3_GRU/20250212-150828
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8659 - loss: 0.2727 - val_accuracy: 0.7690 - val_loss: 0.8529
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9764 - loss: 0.0713 - val_accuracy: 0.7677 - val_loss: 0.9059
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9732 - loss: 0.0620 - val_accuracy: 0.7598 - val_loss: 0.8776
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9782 - loss: 0.0558 - val_accuracy: 0.7677 - val_loss: 0.9690
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9752 - loss: 0.0541 - val_accuracy: 0.7743 - val_loss: 1.0915


In [54]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:20]

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


array([[1.4994219e-01],
       [5.6068337e-01],
       [9.9921966e-01],
       [7.7901028e-02],
       [1.8379193e-04],
       [9.9963641e-01],
       [4.2571014e-01],
       [9.9993348e-01],
       [9.9986243e-01],
       [9.2668599e-01],
       [3.4342389e-04],
       [7.9026687e-01],
       [4.3252262e-04],
       [1.4682512e-01],
       [1.8020060e-04],
       [3.8134251e-03],
       [7.2293315e-04],
       [3.7118065e-04],
       [4.6429224e-02],
       [9.9952590e-01]], dtype=float32)

In [55]:
# Converts model prediction probabilities to label format 
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [56]:
model_3_results = calculate_results(y_true=val_labels,
                                   y_pred=model_3_preds)
model_3_results

{'Accuracy Score ': 77.43,
 'Precision Score': 0.7757380419380466,
 'Recall Score': 0.7742782152230971,
 'F1 Score ': 0.7723566516531356}

In [57]:
## Model 4 Bidirectional RNN
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.GRU(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name='Model_4_Bidirectional')

In [58]:
model_4.summary()

Model: "Model_4_Bidirectional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 15, 128)             │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 128)                 │          74,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,453,441 (5.54 MB)

 Trainable params: 1,453,441 (5.54 MB)

 Non-trainable params: 0 (0.00 B)

In [59]:
# Compile the model 
model_4.compile(loss='binary_crossentropy',
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [60]:
# Fit the model 
model_4_history = model_4.fit(train_sentences,
                             train_labels,
                             epochs=5,
                             validation_data=(val_sentences, val_labels),
                             callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                                   "Model_4_Bidirectional")])

Saving TensorBoard log files to: model_logs/Model_4_Bidirectional/20250212-150837
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9369 - loss: 0.1532 - val_accuracy: 0.7638 - val_loss: 0.9707
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9805 - loss: 0.0438 - val_accuracy: 0.7651 - val_loss: 1.4204
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9793 - loss: 0.0430 - val_accuracy: 0.7572 - val_loss: 1.2789
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9809 - loss: 0.0433 - val_accuracy: 0.7533 - val_loss: 1.6817
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9830 - loss: 0.0357 - val_accuracy: 0.7546 - val_loss: 1.5762


In [61]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:20]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step


array([[6.7375928e-01],
       [7.9183918e-01],
       [9.9998832e-01],
       [2.3850890e-01],
       [1.6491427e-05],
       [9.9997151e-01],
       [9.8763674e-01],
       [9.9999547e-01],
       [9.9999428e-01],
       [9.9978346e-01],
       [7.7528784e-05],
       [9.9951661e-01],
       [3.2708474e-05],
       [2.7296606e-01],
       [2.4077315e-05],
       [1.1456395e-03],
       [1.4525963e-04],
       [1.3807281e-04],
       [2.5033992e-02],
       [9.9997294e-01]], dtype=float32)

In [62]:
# Converts model prediction probabilities to label format 
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [63]:
model_4_results = calculate_results(y_true=val_labels,
                                   y_pred=model_4_preds)
model_4_results

{'Accuracy Score ': 75.46,
 'Precision Score': 0.754186285276044,
 'Recall Score': 0.7545931758530183,
 'F1 Score ': 0.7542315016665913}

In [64]:
import tensorflow_hub as hub
import tensorflow as tf
import tf_keras as keras


sentence_encoder_layer = hub.KerasLayer("https://www.kaggle.com/models/google/universal-sentence-encoder/TensorFlow2/universal-sentence-encoder/2",
                                       input_shape=[],  # Accepts a single sentence as input
                                       dtype=tf.string,  # Input type is string
                                       trainable=False,
                                       name="USE")

In [65]:
# Create the model using tf sequential way 

model_6 = keras.Sequential([
    sentence_encoder_layer,
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(1, activation='sigmoid'),
], name='Model_6_USE')

# Compile the model 
model_6.compile(loss='binary_crossentropy',
               optimizer=keras.optimizers.Adam(),
               metrics=["accuracy"])

model_6.summary()

Model: "Model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense (Dense)               (None, 64)                32832     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [66]:
# Fit the model 
model_6_history = model_6.fit(train_sentences,
                             train_labels,
                             epochs=5,
                             validation_data=(val_sentences, val_labels))
                             #callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                                  # "Model_6_Bidirectional")])

Epoch 1/5
215/215 [==============================] - 6s 12ms/step - loss: 0.5035 - accuracy: 0.7882 - val_loss: 0.4513 - val_accuracy: 0.7966
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.4143 - accuracy: 0.8158 - val_loss: 0.4378 - val_accuracy: 0.8071
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.4007 - accuracy: 0.8227 - val_loss: 0.4338 - val_accuracy: 0.8136
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.3926 - accuracy: 0.8231 - val_loss: 0.4315 - val_accuracy: 0.8097
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.3860 - accuracy: 0.8286 - val_loss: 0.4279 - val_accuracy: 0.8136
